In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Glioblastoma/GSE231994'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Cancer-immune expressional variation in glioblastoma second surgery samples to compare progression and pseudoprogression"
!Series_summary	"We performed cancer-immune gene expression analysis on a case series of glioblastoma second surgery samples due to novel enhancement following chemoradiation that were confirmed based on clinico-pathologic outcome as disease progression (PD) or pseudoprogression (psPD). This was accomplished using an nCounter Pancancer 360 IO panel. Our goals were to (1) determine if psPD events could be distinguished from PD events using differences in immune cell activation versus cancer cell proliferation and (2) examine whether samples stratified based on their molecular profile in the same way as documented clinico-pathologic diagnosis."
!Series_overall_design	"NanoString nCounter pancancer 360 IO panel testing was performed on 48 glioblastoma samples with 27 showing progression (PD) and 21 showing pseudoprogression (psPD)

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 1
age_row = gender_row = None  # set to different values when applicable

def convert_trait(value):
    if 'PD' in value:
        return 1
    elif 'psPD' in value:
        return 0
    else:
        return None

def convert_age(value):
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    if 'Male' in value:
        return 1
    elif 'Female' in value:
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE231994', './preprocessed/Glioblastoma/cohort_info.json', is_gene_available, trait_row is not None)

# If trait_row is not None, extract clinical data
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Glioblastoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Glioblastoma/trait_data/GSE231994.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM7307106': [1], 'GSM7307107': [1], 'GSM7307108': [1], 'GSM7307109': [1], 'GSM7307110': [1], 'GSM7307111': [1], 'GSM7307112': [1], 'GSM7307113': [1], 'GSM7307114': [1], 'GSM7307115': [1], 'GSM7307116': [1], 'GSM7307117': [1], 'GSM7307118': [1], 'GSM7307119': [1], 'GSM7307120': [1], 'GSM7307121': [1], 'GSM7307122': [1], 'GSM7307123': [1], 'GSM7307124': [1], 'GSM7307125': [1], 'GSM7307126': [1], 'GSM7307127': [1], 'GSM7307128': [1], 'GSM7307129': [1], 'GSM7307130': [1], 'GSM7307131': [1], 'GSM7307132': [1], 'GSM7307133': [1], 'GSM7307134': [1], 'GSM7307135': [1], 'GSM7307136': [1], 'GSM7307137': [1], 'GSM7307138': [1], 'GSM7307139': [1], 'GSM7307140': [1], 'GSM7307141': [1], 'GSM7307142': [1], 'GSM7307143': [1], 'GSM7307144': [1], 'GSM7307145': [1], 'GSM7307146': [1], 'GSM7307147': [1], 'GSM7307148': [1], 'GSM7307149': [1], 'GSM7307150': [1], 'GSM7307151': [1], 'GSM7307152': [1], 'GSM7307153': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A2M', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1', 'ALDOA',
       'ALDOC', 'ANGPT1', 'ANGPT2', 'ANGPTL4', 'ANLN', 'APC', 'APH1B', 'API5',
       'APLNR', 'APOE', 'APOL6', 'AQP9'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# Skipping STEP6 as requires_gene_mapping is False

# Proceed to STEP7


Gene annotation preview:
{'ID': ['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1'], 'ORF': ['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 7: Data Normalization and Merging

In [7]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Glioblastoma/gene_data/GSE231994.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Glioblastoma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE231994', './preprocessed/Glioblastoma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Glioblastoma/GSE231994.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Glioblastoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Glioblastoma' in this dataset is severely biased.

